In [ ]:
import pandas as pd

# Load the CSV files
edges_df = pd.read_csv('edges.csv')
nodes_df = pd.read_csv('nodes.csv')

# Standardize text (lowercase and strip spaces) in both edges and nodes to avoid mismatch
edges_df['Source'] = edges_df['Source'].str.lower().str.strip()
edges_df['Target'] = edges_df['Target'].str.lower().str.strip()
nodes_df['Entity'] = nodes_df['Entity'].str.lower().str.strip()

# Fixing Nodes CSV
# Add a unique Id column to the nodes_df
nodes_df['Id'] = range(1, len(nodes_df) + 1)

# Rearrange the columns to match the required format: Id, Label
# Setting 'Entity' as 'Label'
nodes_df['Label'] = nodes_df['Entity']
nodes_df = nodes_df[['Id', 'Label']]

# Create a dictionary to map entity names (Labels) to their corresponding Ids
label_to_id = nodes_df.set_index('Label')['Id'].to_dict()

# Get the next available unique ID
next_id = nodes_df['Id'].max() + 1

# Function to add missing nodes to the nodes_df
def add_node_if_missing(node_label, next_id, nodes_df, label_to_id):
    if node_label not in label_to_id:
        # Add the node to the nodes_df
        new_node = pd.DataFrame({'Id': [next_id], 'Label': [node_label]})
        nodes_df = pd.concat([nodes_df, new_node], ignore_index=True)
        # Add the new entry to the label_to_id dictionary
        label_to_id[node_label] = next_id
        # Increment next_id for the next missing node
        next_id += 1
    return nodes_df, label_to_id, next_id

# Iterate through both Source and Target columns
for index, row in edges_df.iterrows():
    # Check Source column
    source_label = row['Source']
    nodes_df, label_to_id, next_id = add_node_if_missing(source_label, next_id, nodes_df, label_to_id)

    # Check Target column
    target_label = row['Target']
    nodes_df, label_to_id, next_id = add_node_if_missing(target_label, next_id, nodes_df, label_to_id)

# Map the Source and Target columns in edges_df to the correct Ids using the updated label_to_id mapping
edges_df['Source'] = edges_df['Source'].map(label_to_id)
edges_df['Target'] = edges_df['Target'].map(label_to_id)

# Ensure that Source and Target columns are integers (not floats)
edges_df['Source'] = edges_df['Source'].astype(int)
edges_df['Target'] = edges_df['Target'].astype(int)

# Ensure the edges file has the correct columns: Source, Target, Type (assuming default values)
edges_df['Type'] = 'Directed'  # Assuming directed edges
edges_df = edges_df[['Source', 'Target', 'Type', 'Relation']]  # Rearrange to match Gephi's required format

# Check for duplicates
duplicate_nodes = nodes_df[nodes_df.duplicated(subset='Label', keep=False)]

# Output duplicate rows if found
if not duplicate_nodes.empty:
    print("Duplicate nodes found:")
    print(duplicate_nodes)

    # Remove duplicates after checking
    nodes_df = nodes_df.drop_duplicates(subset='Label', keep='first')
    print("Duplicates removed.")
else:
    print("No duplicates found.")

# Save the updated CSV files
nodes_df.to_csv('updated_nodes.csv', index=False)
edges_df.to_csv('updated_edges.csv', index=False)

# Output the fixed DataFrames for verification
print("Updated Nodes DataFrame:")
print(nodes_df)
print("\nUpdated Edges DataFrame:")
print(edges_df)


Duplicate nodes found:
    Id    Label
5    6   france
17  18     apec
18  19     apec
54  55   france
58  59    energ
67  68  generat
73  74    energ
83  84  generat
Duplicates removed.
Updated Nodes DataFrame:
        Id          Label
0        1      singapore
1        2        october
2        3           2021
3        4         russia
4        5       saturday
...    ...            ...
2046  2047             1b
2047  2048  consideration
2048  2049             3a
2049  2050           name
2050  2051       register

[2047 rows x 2 columns]

Updated Edges DataFrame:
      Source  Target      Type    Relation
0        118     119  Directed        skip
1        120     121  Directed          be
2        122       2  Directed       spoke
3        123     124  Directed         set
4        125     126  Directed          is
...      ...     ...       ...         ...
3569     177    1857  Directed   considers
3570    2051     456  Directed  maintained
3571    2051    1455  Directed        

In [2]:
import pandas as pd

# Load the CSV files
edges_df = pd.read_csv('updated_edges.csv')
nodes_df = pd.read_csv('updated_nodes.csv')

duplicate_nodes = nodes_df[nodes_df.duplicated(subset='Label', keep=False)]

# Output duplicate rows if found
if not duplicate_nodes.empty:
    print("Duplicate nodes found:")
    print(duplicate_nodes)

else:
    print("No duplicates found.")

No duplicates found.


# Add Analysis Measures 

In [12]:
import networkx as nx
import pandas as pd

# Load the CSV files
edges_df = pd.read_csv('sentimentEdges.csv')
nodes_df = pd.read_csv('sentimentNodes.csv')


# Create a directed graph from the edge list
G = nx.from_pandas_edgelist(edges_df, source='Source', target='Target', create_using=nx.DiGraph())

# Add node attributes (labels) to the graph
nx.set_node_attributes(G, pd.Series(nodes_df.Label.values, index=nodes_df.Id).to_dict(), 'label')

# Calculate centrality measures
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

# Add centrality values to the node dataframe for display
nodes_df['degree_centrality'] = nodes_df['Id'].map(degree_centrality)
nodes_df['betweenness_centrality'] = nodes_df['Id'].map(betweenness_centrality)


# Display the updated node DataFrame
nodes_df[['Id', 'Label', 'degree_centrality', 'betweenness_centrality']].head()



Id  \
0  649 mw was awarded to large-scale pv plants   
1                          aaprenewable energy   
2                      accelerating renewables   
3        advanced heavy water reactors (ahwrs)   
4                                   architects   

                                         Label  degree_centrality  \
0  649 mw was awarded to large-scale pv plants           0.011858   
1                          aaprenewable energy           0.003953   
2                      accelerating renewables           0.023715   
3        advanced heavy water reactors (ahwrs)           0.007905   
4                                   architects           0.019763   

   betweenness_centrality  
0                     0.0  
1                     0.0  
2                     0.0  
3                     0.0  
4                     0.0

In [13]:
nodes_df.to_csv("sentiment_nodes_centrality.csv", index=False)


# FOR LLM APPROACH

In [2]:
import pandas as pd

# Load the CSV files
sentiment = pd.read_csv('sentiment.csv')
stakeholder = pd.read_csv('stakeholder.csv')
policy = pd.read_csv('policy.csv')

In [ ]:

# Step 1: Create the 'nodes' DataFrame
policy_nodes_df = pd.DataFrame({
    'Id': pd.concat([policy['node_1'], policy['node_2']]).unique(),  # Collect unique node IDs from both node_1 and node_2
    'Label': pd.concat([policy['node_1'], policy['node_2']]).unique()  # Labels are the same as IDs in this case
})

policy_edges_df = pd.DataFrame({
    'Source': policy['node_1'],  # Source is node_1
    'Target': policy['node_2'],  # Target is node_2
    'Weight': policy['count'],   # Weight is based on the 'count' column
    'Type': 'Directed',
    'Relation': policy['edge']
})

# Step 3: Save to Excel files
nodes_file = 'policyNodes.csv'
edges_file = 'policyEdges.csv'

# Save the nodes and edges as Excel files
policy_nodes_df.to_csv(nodes_file, index=False)
policy_edges_df.to_csv(edges_file, index=False)

policy_nodes_df, policy_edges_df  # Return paths to the saved files


In [7]:

# Step 1: Create the 'nodes' DataFrame
stakeholder_nodes_df = pd.DataFrame({
    'Id': pd.concat([stakeholder['node_1'], stakeholder['node_2']]).unique(),  # Collect unique node IDs from both node_1 and node_2
    'Label': pd.concat([stakeholder['node_1'], stakeholder['node_2']]).unique()  # Labels are the same as IDs in this case
})


# Step 2: Create the 'edges' DataFrame
stakeholder_edges_df = pd.DataFrame({
    'Source': stakeholder['node_1'],  # Source is node_1
    'Target': stakeholder['node_2'],  # Target is node_2
    'Weight': stakeholder['count'],   # Weight is based on the 'count' column
    'Type': 'Directed',
    'Relation': stakeholder['edge']
})


# Step 3: Save to Excel files
nodes_file = 'stakeholderNodes.csv'
edges_file = 'stakeholderEdges.csv'

# Save the nodes and edges as Excel files
stakeholder_nodes_df.to_csv(nodes_file, index=False)
stakeholder_edges_df.to_csv(edges_file, index=False)



In [8]:

# Step 1: Create the 'nodes' DataFrame
sentiment_nodes_df = pd.DataFrame({
    'Id': pd.concat([sentiment['node_1'], sentiment['node_2']]).unique(),  # Collect unique node IDs from both node_1 and node_2
    'Label': pd.concat([sentiment['node_1'], sentiment['node_2']]).unique()  # Labels are the same as IDs in this case
})

# Step 2: Create the 'edges' DataFrame
sentiment_edges_df = pd.DataFrame({
    'Source': sentiment['node_1'],  # Source is node_1
    'Target': sentiment['node_2'],  # Target is node_2
    'Weight': sentiment['count'],   # Weight is based on the 'count' column
    'Type': 'Directed',
    'Relation': sentiment['edge']
})

# Step 3: Save to Excel files
nodes_file = 'sentimentNodes.csv'
edges_file = 'sentimentEdges.csv'

# Save the nodes and edges as Excel files
sentiment_nodes_df.to_csv(nodes_file, index=False)
sentiment_edges_df.to_csv(edges_file, index=False)
